In [1]:
import numpy as np
import matplotlib.pyplot as plt 
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.ndimage import gaussian_filter
plt.style.use('sensitivity.mplstyle')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [2]:
G4Offset = -894.59

minR = 0.0
maxR = 627.0458
minZ = -767.883544921875 + 3.0 + G4Offset + 1581
maxZ = 513.116455078125 + G4Offset + 1581

with open('/gpfs/slac/staas/fs1/g/exo/exo_data8/exo_data/users/akojamil/chroma/sensitivity2020/20200528_sensitivity_lightmap_raw_data.npy', 'rb') as f:
    Radius = np.load(f)
    Height = np.load(f)
    Eff = np.load(f)
Height = Height + G4Offset + 1581

cut0 = np.where(Eff > 0.0)
Radius = Radius[cut0]
Height = Height[cut0]
Eff = Eff[cut0]

FileNotFoundError: [Errno 2] No such file or directory: '/gpfs/slac/staas/fs1/g/exo/exo_data8/exo_data/users/akojamil/chroma/sensitivity2020/20200528_sensitivity_lightmap_raw_data.npy'

In [ ]:
bins = 0.5

maxR = np.around(maxR, 1)
minZ = np.around(minZ, -1)
maxZ = np.around(maxZ, -1)

binR = np.linspace(0.0,maxR,int(maxR/float(bins)))
binZ = np.linspace(minZ,maxZ,int((maxZ-minZ)/float(bins)))

In [ ]:
HistC, XEdges, YEdges =  np.histogram2d(x=Radius, y=Height, bins=(binR, binZ))
HistEff, XEdges, YEdges =  np.histogram2d(x=Radius, y=Height, weights=Eff, bins=(binR, binZ))

X, Y = np.meshgrid(XEdges, YEdges)

HistC[HistC==0] = 1.0 # prevent division by zero

Hist = (HistEff/HistC).T

# replace empty bin with average over adjacent 8 bins
Cut = np.where((X[:-1,:-1]<550) & (Y[:-1,:-1]<1150) & (Y[:-1,:-1]>0) & (Hist==0))
XCut = Cut[0]
YCut = Cut[1]
for x,y in zip(XCut,YCut):
    HistMean = np.mean([Hist[x+1,y], Hist[x-1,y], Hist[x,y+1], Hist[x,y-1], Hist[x+1,y+1], Hist[x-1,y-1], Hist[x+1,y-1], Hist[x-1,y+1]])
    Hist[x,y] = HistMean

Hist = gaussian_filter(Hist, sigma=1.0)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 12))    

plt.xlabel('Radius [mm]')
plt.ylabel('Z [mm]')

plt.xlim(0, maxR)
plt.ylim(minZ, maxZ)

im = plt.imshow(Hist, origin='lower', aspect='auto', extent=[minR, maxR, minZ, maxZ], cmap='viridis')

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
cb = fig.colorbar(im, cax=cax, orientation='vertical')
cb.ax.tick_params(labelsize=16)
cb.set_label(label=r'Photon Transport Efficiency [$\%$]', fontsize=16)
im.set_clim(vmin=0.0, vmax=50.0)

plt.savefig('lightmap.png')
plt.show()

In [27]:
import uproot as up
import pandas as pd

thisfile = up.open('/usr/gapps/nexo/nexo-offline/data/20200528_lightmap_0.50_0.50.root')
myhist = thisfile['lightmap']


In [5]:
histnp = myhist.numpy()

In [8]:
edges = histnp[0]
vals = histnp[1]

In [21]:
print(len(edges[0]))
print(len(vals))

2560
1


In [22]:
vals

[(array([0.000e+00, 5.000e-01, 1.000e+00, ..., 6.260e+02, 6.265e+02,
         6.270e+02]),
  array([-1660. , -1659.5, -1659. , ...,  -381. ,  -380.5,  -380. ]))]

In [24]:
data = myhist.pandas()

In [25]:
data.head()

count  variance
[-inf, 0.0) [-inf, -1660.0)       0.0       0.0
            [-1660.0, -1659.5)    0.0       0.0
            [-1659.5, -1659.0)    0.0       0.0
            [-1659.0, -1658.5)    0.0       0.0
            [-1658.5, -1658.0)    0.0       0.0

In [29]:
newdata = data.unstack()

In [40]:
print(newdata['count'].head())

             [-inf, -1660.0)  [-1660.0, -1659.5)  [-1659.5, -1659.0)  \
[-inf, 0.0)              0.0            0.000000            0.000000   
[0.0, 0.5)               0.0            1.171105            1.184282   
[0.5, 1.0)               0.0            1.166797            1.173705   
[1.0, 1.5)               0.0            1.171014            1.169253   
[1.5, 2.0)               0.0            1.176571            1.169740   

             [-1659.0, -1658.5)  [-1658.5, -1658.0)  [-1658.0, -1657.5)  \
[-inf, 0.0)            0.000000            0.000000            0.000000   
[0.0, 0.5)             1.196720            1.201756            1.201591   
[0.5, 1.0)             1.180489            1.187698            1.198081   
[1.0, 1.5)             1.172541            1.184728            1.197689   
[1.5, 2.0)             1.172188            1.187298            1.198075   

             [-1657.5, -1657.0)  [-1657.0, -1656.5)  [-1656.5, -1656.0)  \
[-inf, 0.0)            0.000000          

In [38]:
newdata['count'].loc[1]

[-inf, -1660.0)       0.000000e+00
[-1660.0, -1659.5)    1.171014e+00
[-1659.5, -1659.0)    1.169253e+00
[-1659.0, -1658.5)    1.172541e+00
[-1658.5, -1658.0)    1.184728e+00
[-1658.0, -1657.5)    1.197689e+00
[-1657.5, -1657.0)    1.208059e+00
[-1657.0, -1656.5)    1.208384e+00
[-1656.5, -1656.0)    1.204124e+00
[-1656.0, -1655.5)    1.207690e+00
[-1655.5, -1655.0)    1.215693e+00
[-1655.0, -1654.5)    1.219773e+00
[-1654.5, -1654.0)    1.224070e+00
[-1654.0, -1653.5)    1.230199e+00
[-1653.5, -1653.0)    1.237711e+00
[-1653.0, -1652.5)    1.242646e+00
[-1652.5, -1652.0)    1.240775e+00
[-1652.0, -1651.5)    1.240553e+00
[-1651.5, -1651.0)    1.245190e+00
[-1651.0, -1650.5)    1.247623e+00
[-1650.5, -1650.0)    1.248637e+00
[-1650.0, -1649.5)    1.247760e+00
[-1649.5, -1649.0)    1.248130e+00
[-1649.0, -1648.5)    1.251933e+00
[-1648.5, -1648.0)    1.257785e+00
[-1648.0, -1647.5)    1.261871e+00
[-1647.5, -1647.0)    1.263959e+00
[-1647.0, -1646.5)    1.263286e+00
[-1646.5, -1646.0)  